---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [31]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    hand = open('university_towns.txt')
    df = []
    numlist = list()
    for line in hand:
        line = line.rstrip()
        if(line[-6:] == '[edit]'):
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            df.append([state,town])
        else:
            town = line
            df.append([state,town])
    
    df1 = pd.DataFrame(df,columns = ['State','RegionName'])
    return df1   
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [52]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
     df.iloc[iter][1] > df.iloc[iter+1][1] and df.iloc[iter+1][1] > df.iloc[iter+2][1] and df.iloc[iter+2][1] < df.iloc[iter+3][1] and df.iloc[iter+3][1] < df.iloc[iter+4][1]:'''
    
    df = pd.read_excel("gdplev.xls")
    df = df[7:286]
    df.drop(df.columns[0:4], axis=1, inplace =True)
    df.drop(df.columns[3], axis=1, inplace =True)
    result = []
    print(df.iloc[2][1])
    for iter in range(df.shape[0]):
        if (iter+3)>df.shape[0]:
            return result
        elif df.iloc[iter][2] > df.iloc[iter+1][2] and df.iloc[iter+1][2] > df.iloc[iter+2][2] :
            result.append(df.iloc[iter][0])

df = get_recession_start()

df



250.1


['1947q1',
 '1948q4',
 '1953q2',
 '1953q3',
 '1957q3',
 '1969q3',
 '1974q2',
 '1974q3',
 '1980q1',
 '1981q3',
 '1990q3',
 '2008q2',
 '2008q3',
 '2008q4']

In [45]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    df = df[7:286]
    df.drop(df.columns[0:4], axis=1, inplace =True)
    df.drop(df.columns[3], axis=1, inplace =True)
    result = []
    print(df.iloc[2][1])
    for iter in range(df.shape[0]):
        if (iter+3)>df.shape[0]:
            return result
        elif df.iloc[iter][2] > df.iloc[iter+1][2] and df.iloc[iter+1][2] > df.iloc[iter+2][2] and df.iloc[iter+2][2] < df.iloc[iter+3][2] and df.iloc[iter+3][2] < df.iloc[iter+4][2]:
            result.append(df.iloc[iter+4][0])
get_recession_end()       

250.1


['1948q1',
 '1954q3',
 '1958q3',
 '1970q3',
 '1975q3',
 '1981q1',
 '1991q3',
 '2009q4']

In [46]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls")
    df = df[7:286]
    df.drop(df.columns[0:4], axis=1, inplace =True)
    df.drop(df.columns[3], axis=1, inplace =True)
    result = []
    print(df.iloc[2][1])
    for iter in range(df.shape[0]):
        if (iter+3)>df.shape[0]:
            return result
        elif df.iloc[iter][2] > df.iloc[iter+1][2] and df.iloc[iter+1][2] > df.iloc[iter+2][2] and df.iloc[iter+2][2] < df.iloc[iter+3][2] and df.iloc[iter+3][2] < df.iloc[iter+4][2]:
            result.append(df.iloc[iter+2][0])
get_recession_bottom()

250.1


['1947q3',
 '1954q1',
 '1958q1',
 '1970q1',
 '1975q1',
 '1980q3',
 '1991q1',
 '2009q2']

In [28]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    dict = {'01':'q1','04':'q2','07':'q3','10':'q4'}
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    df.drop(df.columns[6:63], axis=1, inplace =True)
    df1 = df[['RegionName', 'State']]
    cols=df.columns
    while iter < 192:
        
        col_name= cols[iter][0:cols[iter].find('-')]
        quarter_name = dict[cols[iter][cols[iter].find('-')+1:]]
        col_name = col_name+quarter_name
        df1[col_name] = df[[cols[iter],cols[iter+1],cols[iter+2]]].mean(axis=1)
                         
        iter=iter+3
        
    df1['2016q3']= df[[cols[iter],cols[iter+1]]].mean(axis=1)
    df1['2016q3'] = (2 * df1['2016q3'])/3
    
    df1['State'] = df1['State'].map(states)
    df1.set_index(['State','RegionName'],inplace=True)

    return df1

Index(['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '2001-01', '2001-02', '2001-03', '2001-04',
       ...
       '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
       '2016-05', '2016-06', '2016-07', '2016-08'],
      dtype='object', length=194)


C:\Users\VIBIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\VIBIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\VIBIN\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

,,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,3.914667e+05
California,Los Angeles,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,2.870000e+05,3.022333e+05,3.189000e+05,3.345000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,3.893667e+05
Illinois,Chicago,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,1.789667e+05,1.837000e+05,1.871000e+05,1.943000e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,1.413333e+05
Pennsylvania,Philadelphia,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,6.216667e+04,6.350000e+04,6.473333e+04,6.620000e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,8.580000e+04
Arizona,Phoenix,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,1.305333e+05,1.332333e+05,1.357000e+05,1.379333e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,1.301333e+05
Nevada,Las Vegas,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,1.523333e+05,1.554000e+05,1.594667e+05,1.626667e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,1.333000e+05
California,San Diego,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,3.311000e+05,3.507000e+05,3.664333e+05,3.774333e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,3.598333e+05
Texas,Dallas,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,9.540000e+04,9.650000e+04,9.820000e+04,9.850000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,9.953333e+04
California,San Jose,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,4.732333e+05,4.786000e+05,4.782667e+05,4.737000e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,5.481333e+05


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"